## 3.6 Homework

Previous homeworks:

**Week 1:**
* https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/01-intro/homework.md

**Week 2:**
* https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/02-experiment-tracking/homework.md

The goal of this homework is to familiarize users with workflow orchestration. We start from the solution of homework 1. The notebook can be found below:

https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/01-intro/homework.ipynb

This has already been converted to a script called `homework.py` in the `03-orchestration` folder of this repo.

## Motivation

We already have a model training script. Maybe a data scientist in your team handed it to you and your job is schedule the running of training script using a workflow orchestration - Prefect in this case. Below are the requirements. Do not implement them yet, we will do so in this exercise. Just understand the goal.

1. The training flow will be run every month.
2. The flow will take in a parameter called `date` which will be a datetime.
    a. `date` should default to None
    b. If `date` is None, set `date` as the current day. Use the data from 2 months back as the training data and the data from the previous month as validation data.
    c. If `date` is passed, get 2 months before the `date` as the training data, and the previous month as validation data.
    d. As a concrete example, if the date passed is "2021-03-15", the training data should be "fhv_tripdata_2021-01.parquet" and the validation file will be "fhv_trip_data_2021-02.parquet"
3. Save the model as "model-{date}.pkl" where date is in `YYYY-MM-DD`. Note that `date` here is the value of the flow `parameter`. In practice, this setup makes it very easy to get the latest model to run predictions because you just need to get the most recent one.
4. In this example we use a DictVectorizer. That is needed to run future data through our model. Save that as "dv-{date}.pkl". Similar to above, if the date is `2021-03-15`, the files output should be `model-2021-03-15.bin` and `dv-2021-03-15.b`.

This convention is not strict in industry, and in practice, you will come up with your own system to manage these training pipeline runs. For example, if we wanted to train on the whole history instead of just one month, we'd need to allow for added parameterization and logic in our flow. If the data came in weekly instead of monthly, we might need a different naming convention. But these requirements are already a simple approximation of something you could use in production.

On the deployment side, it's very easy to just pull in the latest data and predict it using the latest model and vectorizer files. Tools the MLFlow in the last chapter can simplify that process as well. This homework will focus more on the batch training.

In order, this homework assignment will be about:

1. Converting the script to a Flow
2. Changing the parameters to take in a `date`. Making this parameter dynamic.
3. Scheduling a batch training job that outputs the latest model somewhere

## Setup

You can use either local Prefect Orion or a VM hosted Prefect Orion instance for this. It shouldn't matter. Just note that if you use a VM hosted one, you will need to configure your local API to hit the VM.

Video 3.4 of the course will give more detailed instructions if you been run it on a VM.

## Get the data

In [1]:
# use brace expansion to download all January - March files
!wget -P data 'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-0'{1..3}'.parquet'

--2022-06-04 23:19:14--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.192.161
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.192.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘data/fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  2.75MB/s    in 5.6s    

2022-06-04 23:19:20 (2.04 MB/s) - ‘data/fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]

--2022-06-04 23:19:20--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Reusing existing connection to nyc-tlc.s3.amazonaws.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘data/fhv_tripdata_2021-02.parquet’

fhv_tripdata_2021-0 100%[===================>]  10.15M  5.85MB/s    in 1.7s    

2022-06-04 23:19:22 (5.85 


## Q1. Converting the script to a Prefect flow

If you want to follow the videos exactly, do:

```
pip install prefect==2.0b5
```

If you need Windows support, check `homework-windows.md` for installation instructions.

The current script `homework.py` is a fully functional script as long as you already have `fhv_trip_data_2021-01.parquet` and `fhv_trip_data_2021-02.parquet` inside a `data` folder. You should be able to already run it using:

```
python homework.py
```

We want to bring this to workflow orchestration to add observability around it. The `main` function will be converted to a `flow` and the other functions will be `tasks`. After adding all of the decorators, there is actually one task that you will need to call `.result()` for inside the `flow` to get it to work. Which task is this?

* read_data
* prepare_features
* train_model
* run_model

Important: change all `print` statements to use the Prefect logger. Using the `print` statement will not appear in the Prefect UI. You have to call `get_run_logger` at the start of the task to use it.


In [2]:
!python homework.py

20:51:28.293 | INFO    | prefect.engine - Created flow run 'chirpy-mackerel' for flow 'main'
20:51:28.293 | INFO    | Flow run 'chirpy-mackerel' - Using task runner 'SequentialTaskRunner'
20:51:28.300 | WARNING | Flow run 'chirpy-mackerel' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
20:51:28.339 | INFO    | Flow run 'chirpy-mackerel' - Created task run 'read_data-4c7f9de4-0' for task 'read_data'
20:51:30.890 | INFO    | Task run 'read_data-4c7f9de4-0' - Finished in state Completed()
20:51:30.912 | INFO    | Flow run 'chirpy-mackerel' - Created task run 'prepare_features-4ee39d9f-0' for task 'prepare_features'
20:51:30.999 | INFO    | Task run 'prepare_features-4ee39d9f-0' - The mean duration of training is 16.2472533682457
20:51:34.467 | INFO    | Task run 'prepare_features-4ee39d9f-0' - Finished in state Completed()
20:51:34.513 | INFO    | Flow run 'chirpy-mackerel' - Created task run

After adding all of the decorators, there is actually one task that you will need to call `.result()` for inside the `flow` to get it to work. Which task is this?

--> train_model
since the call to this task is unpacking into lr, dv = ...